In [1]:
import spacy
from spacy import displacy
import numpy as np
import pandas as pd
import random
import re
import operator
import math
import itertools
import json
from spacy.util import minibatch, compounding
from spacy.training import Example
from spacy.tokens import Doc
from pathlib import Path
from spacy.matcher import Matcher

nlp = spacy.load('en_core_web_md')


ner = nlp.get_pipe('ner')

# Load excel into pandas dataframe

In [2]:
training = pd.ExcelFile('NER training.xlsx')
df_training = training.parse('Sheet1')
df_training

Query  Intent  \
0              Time is 2030 and the date is 20/08/2030     NaN   
1                         Set a reminder at 1500 later     NaN   
2                                        Today at 0830     NaN   
3                                 Print report at 1900     NaN   
4    manufacturing facilitation efficiency of ab380...     NaN   
..                                                 ...     ...   
122              MTBF of EAGLE AR increase by how much     NaN   
123  which IDEALMOLD oee increase more than 20% fro...     NaN   
124                 number of ms90 tst less than 30min     NaN   
125          number of ps410 with tsdt more than 20min     NaN   
126         number of tlb139 with tpt more than 2hours     NaN   

                                    <ATTR>  <LOT>  <EMPLOYEE>  <RECIPE>  \
0                                      NaN    NaN         NaN       NaN   
1                                      NaN    NaN         NaN       NaN   
2                                      NaN    NaN         NaN       NaN   
3                                      NaN    NaN         NaN       NaN   
4    manufacturing facilitation efficiency    NaN         NaN       NaN   
..                                     ...    ...         ...       ...   
122                                   MTBF    NaN         NaN       NaN   
123                                    oee    NaN         NaN       NaN   
124                                    tst    NaN         NaN       NaN   
125                                   tsdt    NaN         NaN       NaN   
126                                    tpt    NaN         NaN       NaN   

     <TOOL>  <MACHINE>       <TEMPORAL>          <DATE>  <Rel-DATE>  \
0       NaN        NaN  2030|20/08/2030      20/08/2030         NaN   
1       NaN        NaN             1500             NaN         NaN   
2       NaN        NaN       0830|Today           today         NaN   
3       NaN        NaN             1900             NaN         NaN   
4       NaN      ab380  20/03/2021|1730      20/03/2021         NaN   
..      ...        ...              ...             ...         ...   
122     NaN   EAGLE AR              NaN             NaN         NaN   
123     NaN  IDEALMOLD   monday|tuesday  monday|tuesday         NaN   
124     NaN       ms90              NaN             NaN         NaN   
125     NaN      ps410              NaN             NaN         NaN   
126     NaN     tlb139              NaN             NaN         NaN   

     <Rel-TIME>  <Def-TIME>  <Def-DATE> <TIME>     <DESC> <DIRECTION>  \
0           NaN         NaN         NaN   2030        NaN         NaN   
1           NaN         NaN         NaN   1500        NaN         NaN   
2           NaN         NaN         NaN   0830        NaN         NaN   
3           NaN         NaN         NaN   1900        NaN         NaN   
4           NaN         NaN         NaN   1730        NaN         NaN   
..          ...         ...         ...    ...        ...         ...   
122         NaN         NaN         NaN    NaN        NaN    increase   
123         NaN         NaN         NaN    NaN  more than    increase   
124         NaN         NaN         NaN    NaN  less than         NaN   
125         NaN         NaN         NaN    NaN  more than         NaN   
126         NaN         NaN         NaN    NaN  more than         NaN   

    <MEASUREMENT>  <QUANTITY>  
0             NaN         NaN  
1             NaN         NaN  
2             NaN         NaN  
3             NaN         NaN  
4             NaN         NaN  
..            ...         ...  
122           NaN         NaN  
123           20%         NaN  
124         30min         NaN  
125         20min         NaN  
126        2hours         NaN  

[127 rows x 19 columns]

In [3]:
pd.set_option('display.max_rows', None)

# Clean data

In [4]:
df_training = df_training.replace(np.nan, '')
df_training

35        DS209          last week         last week                         
36        iHAWK          yesterday         yesterday                         
37      is600gs          yesterday         yesterday                         
38      is600gs                                                              
39         ms60          yesterday         yesterday                         
40         AD90              today             today                         
41         ad90                                                              
42        RO832          last week         last week                         
43       ihb139          yesterday         yesterday                         
44       SD832D              today             today                         
45        stk40                                                              
46        stk40          yesterday         yesterday                         
47        ab380          last week         last week                         
48         AD90                                                              
49        ab380                                                              
50      3rdbase                                                              
51        AB350                                                              
52        AB380                                                              
53        AB339                                                              
54        ab383                                                              
55        ab339                                                              
56   ab350vlamp                                                              
57   ab350vlamp                                                              
58        ab339                                                              
59      3rdbase                                                              
60      ab350vm                                                              
61      ab350vs                                                              
62      ab383dh                                                              
63      ab383dh                                                              
64        ad210                                                              
65    ab280plus                                                              
66        ad280                                                              
67        ab380                                                              
68      ab383dh                                                              
69   ab350vlamp                                                              
70    ad210plus                                                              
71        ad211                                                              
72         ad50                                                              
73      ab350vs                                                              
74      3rdbase                                                              
75    2d211plus                                                              
76        ad280                                                              
77      ab383dh                                                              
78      ab350vm                                                              
79     ad8312fc                                                              
80     ad50plus                                                              
81        ad828                                                              
82        ad830                                                              
83       ad8312                                                              
84       ad830l                                                              
85       ad830r                                                              
86     ad8312fc       

# Helper methods to get the index of each time of entity
## This step is done to prepare the training data

In [5]:
def get_ATTR(Query, ATTR):

    start = 0
    end = 0
    attr_to_find = r'\b{}\b'.format(ATTR.lower())

    match = re.search(attr_to_find, Query.lower())
    if match:
        start = match.start()
        end = match.end()
        ent_tuple = (start, end, "<ATTR>")
        return ent_tuple

# def get_TEMP(Query, TEMP):

#     start = 0
#     end = 0
#     temp_to_find = r'\b{}\b'.format(TEMP.lower())

#     match = re.search(temp_to_find, Query.lower())

#     if match:
#         start = match.start()
#         end = match.end()
#         ent_tuple = (start, end, "<TEMP>")
#         # print(ent_tuple)
#         return ent_tuple

def get_DATE(Query, DATE):
    start = 0
    end = 0
    date_to_find = r'\b{}\b'.format(DATE.lower())

    match = re.search(date_to_find, Query.lower())
    if match:
        start = match.start()
        end = match.end()
        ent_tuple = (start, end, "<DATE>")
        return ent_tuple

def get_TIME(Query, TIME):

    start = 0
    end = 0
    time_to_find = r'\b{}\b'.format(TIME.lower())

    match = re.search(time_to_find, Query.lower())
    if match:
        start = match.start()
        end = match.end()
        ent_tuple = (start, end, "<TIME>")
        return ent_tuple

def get_MACHINE(Query, MACHINE):
    start = 0
    end = 0
    mach_to_find = r'\b{}\b'.format(MACHINE.lower())
    match = re.search(mach_to_find, Query.lower())
    if match:
        start = match.start()
        end = match.end()
        ent_tuple = (start, end, "<MACHINE>")
        return ent_tuple

def get_DESC(Query, OBJ):
    start = 0
    end = 0
    desc_to_find = r'\b{}\b'.format(OBJ.lower())

    match = re.search(desc_to_find, Query.lower())
    if match:
        start = match.start()
        end = match.end()
        ent_tuple = (start, end, "<DESC>")
        return ent_tuple

def get_DIR(Query, OBJ):
    start = 0
    end = 0
    dir_to_find = r'\b{}\b'.format(OBJ.lower())

    match = re.search(dir_to_find, Query.lower())
    if match:
        start = match.start()
        end = match.end()
        ent_tuple = (start, end, "<DIRECTION>")
        return ent_tuple

def get_MEASUREMENT(Query, MEASUREMENT):
    start = 0
    end = 0
    measurement_to_find = r'\b{}\b'.format(MEASUREMENT.lower())

    match = re.search(measurement_to_find, Query.lower())
    if match:
        start = match.start()
        end = match.end()
        ent_tuple = (start, end, "<MEASUREMENT>")
        return ent_tuple

def get_QUANTITY(Query, QUANTITY):
    start = 0
    end = 0
    quantity_to_find = r'\b{}\b'.format(str(math.trunc(int(float(QUANTITY.lower())))))

    match = re.search(quantity_to_find, Query.lower())
    if match:
        start = match.start()
        end = match.end()
        ent_tuple = (start, end, "<QUANTITY>")
        return ent_tuple
# def get_DATE():

# def get_TIME():


def get_ATTR_MACHINE_TIME_DATE_DESC(Query, ATTR, MACHINE, DATE, TIME, DESC, MEASUREMENT):
    ent_list = []

    if len(ATTR) != 0:
        for a in ATTR:
            ent_list.append(get_ATTR(Query, a))
    
    for d in DATE:   
        ent_list.append(get_DATE(Query, d))
    for t in TIME:   
        ent_list.append(get_TIME(Query, t))
    for o in MACHINE:
        ent_list.append(get_MACHINE(Query, o))
    for d in DESC:
        ent_list.append(get_DESC(Query,d))
    for m in MEASUREMENT:
        ent_list.append(get_MEASUREMENT(Query,m))
    for q in QUANTITY:
        ent_list.append(get_QUANTITY(Query,q))
        # ent_list.append(get_(Query, ATTR))
    # ent_list.append(get_)
    # print(ent_list)
    return ent_list



# Build training data from the dataframe by calling helper methods

In [6]:
training_data = []

for i in df_training.index:
    ent = []

    Query = df_training['Query'][i]
    Query = Query.replace(u'\xa0', u' ')


    ATTR = str(df_training['<ATTR>'][i])
    if ATTR != '':
        ATTR = ATTR.replace(u'\xa0', u' ')
        list_of_attr = ATTR.split('|')
        for a in list_of_attr:
            ent.append(get_ATTR(Query, a))

    MACH = str(df_training['<MACHINE>'][i])
    if MACH != '':
        MACH = MACH.replace(u'\xa0', u' ')
        list_of_mach = MACH.split('|')
        for o in list_of_mach:
            ent.append(get_MACHINE(Query, o))
r(df_training['<TIME>'][i])
    if TIME != '':
        TIME = TIME.replace(u'\xa0', u' ')
        list_of_time = TIME.split('|')
        for t in list_of_time:
            ent.append(get_TIME(Query, t))

    DESC = str(df_training['<DESC>'][i])
    if DESC != '':
        DESC = DESC.replace(u'\xa0', u' ')
        list_of_desc = DESC.split('|')
        for d in list_of_desc:
            ent.append(get_DESC(Query, d))

    DIRECTION = str(df_training['<DIRECTION>'][i])
    if DIRECTION != '':
        DIRECTION = DIRECTION.replace(u'\xa0', u' ')
        list_of_dir = DIRECTION.split('|')
        for dir in list_of_dir:
            ent.append(get_DIR(Query, dir))
    
    MEASUREMENT = str(df_training['<MEASUREMENT>'][i])
    if MEASUREMENT != '':
        MEASUREMENT = MEASUREMENT.replace(u'\xa0', u' ')
        list_of_measurement = MEASUREMENT.split('|')
        for m in list_of_measurement:
            ent.append(get_MEASUREMENT(Query, m))

    QUANTITY = str(df_training['<QUANTITY>'][i])
    if QUANTITY != '':
        QUANTITY = QUANTITY.replace(u'\xa0', u' ')
        list_of_quantity = QUANTITY.split('|')
        for q in list_of_quantity:
            ent.append(get_QUANTITY(Query, q))


    ent_dict = {"entities": ent}
    training_data.append((Query, ent_dict))
    
    # TEMP = str(df_training['<TEMPORAL>'][i])
    # OBJ = str(df_training['<OBJ>'][i])
    # DATE = str(df_training['<DATE>'][i])
    # TIME = str(df_training['<TIME>'][i])
for i in training_data:
    print(i)

('Time is 2030 and the date is 20/08/2030', {'entities': [(29, 39, '<DATE>'), (8, 12, '<TIME>')]})
('Set a reminder at 1500 later', {'entities': [(18, 22, '<TIME>')]})
('Today at 0830', {'entities': [(0, 5, '<DATE>'), (9, 13, '<TIME>')]})
('Print report at 1900', {'entities': [(16, 20, '<TIME>')]})
('manufacturing facilitation efficiency of ab380 on 20/03/2021 at 1730', {'entities': [(0, 37, '<ATTR>'), (41, 46, '<MACHINE>'), (50, 60, '<DATE>'), (64, 68, '<TIME>')]})
('What is the total engineering time of EAGLE60 on 20/06/1997', {'entities': [(12, 34, '<ATTR>'), (38, 45, '<MACHINE>'), (49, 59, '<DATE>')]})
('Current time is 12:00', {'entities': [(16, 21, '<TIME>')]})
('Meet me at 23:20', {'entities': [(11, 16, '<TIME>')]})
('I will be free at 17:40', {'entities': [(18, 23, '<TIME>')]})
('2 Feb 2021', {'entities': [(0, 10, '<DATE>')]})
('2 August 2023', {'entities': [(0, 13, '<DATE>')]})
('2 September 2021', {'entities': [(0, 16, '<DATE>')]})
('20 Mar', {'entities': [(0, 6, '<DATE>')]})

# NOTE: THERE MIGHT BE FORMATTING ISSUES WITH % SIGNS
# I had to manually edit the 'NONE' tags to '<MEASUREMENT>' tags
## Below is the final training data

In [7]:
TRAINING_DATA = [
    ('Time is 2030 and the date is 20/08/2030', {'entities': [(29, 39, '<DATE>'), (8, 12, '<TIME>')]}),
('Set a reminder at 1500 later', {'entities': [(18, 22, '<TIME>')]}),
('Today at 0830', {'entities': [(0, 5, '<DATE>'), (9, 13, '<TIME>')]}),
('Print report at 1900', {'entities': [(16, 20, '<TIME>')]}),
('manufacturing facilitation efficiency of ab380 on 20/03/2021 at 1730', {'entities': [(0, 37, '<ATTR>'), (41, 46, '<MACHINE>'), (50, 60, '<DATE>'), (64, 68, '<TIME>')]}),
('What is the total engineering time of EAGLE60 on 20/06/1997', {'entities': [(12, 34, '<ATTR>'), (38, 45, '<MACHINE>'), (49, 59, '<DATE>')]}),
('Current time is 12:00', {'entities': [(16, 21, '<TIME>')]}),
('Meet me at 23:20', {'entities': [(11, 16, '<TIME>')]}),
('I will be free at 17:40', {'entities': [(18, 23, '<TIME>')]}),
('2 Feb 2021', {'entities': [(0, 10, '<DATE>')]}),
('2 August 2023', {'entities': [(0, 13, '<DATE>')]}),
('2 September 2021', {'entities': [(0, 16, '<DATE>')]}),
('20 Mar', {'entities': [(0, 6, '<DATE>')]}),
('June 22', {'entities': [(0, 7, '<DATE>')]}),
('today is 21 Dec', {'entities': [(0, 5, '<DATE>'), (9, 15, '<DATE>')]}),
('2003 May 8', {'entities': [(0, 10, '<DATE>')]}),
('0930 on 2003 April 4', {'entities': [(8, 20, '<DATE>'), (0, 4, '<TIME>')]}),
('Jun 22', {'entities': [(0, 6, '<DATE>')]}),
('What is output from 11/03 to 11/04', {'entities': [(8, 14, '<ATTR>'), (20, 25, '<DATE>'), (29, 34, '<DATE>')]}),
('last week', {'entities': [(0, 9, '<DATE>')]}),
('last month', {'entities': [(0, 10, '<DATE>')]}),
('what is the date last monday', {'entities': [(17, 28, '<DATE>')]}),
('output of AB350 yesterday', {'entities': [(0, 6, '<ATTR>'), (10, 15, '<MACHINE>'), (16, 25, '<DATE>')]}),
('MTBA of FT2030S last week', {'entities': [(0, 4, '<ATTR>'), (8, 15, '<MACHINE>'), (16, 25, '<DATE>')]}),
('what is TUDT of HP229 on 1 may', {'entities': [(8, 12, '<ATTR>'), (16, 21, '<MACHINE>'), (25, 30, '<DATE>')]}),
('total standby time of IHAWK yesterday', {'entities': [(0, 18, '<ATTR>'), (22, 27, '<MACHINE>'), (28, 37, '<DATE>')]}),
('22 june number of assists for ISLINDA', {'entities': [(8, 25, '<ATTR>'), (30, 37, '<MACHINE>'), (0, 7, '<DATE>')]}),
('mtba for GW APP in the last 2 hours', {'entities': [(0, 4, '<ATTR>'), (9, 15, '<MACHINE>'), (23, 35, '<TIME>')]}),
('total productive time of LOTUS-E 5 hours ago', {'entities': [(0, 21, '<ATTR>'), (25, 32, '<MACHINE>'), (33, 44, '<TIME>')]}),
('Number of assists in the last 30 mins for is600gs', {'entities': [(0, 17, '<ATTR>'), (42, 49, '<MACHINE>'), (25, 37, '<TIME>')]}),
('what is the increase in mtba for AEROLED', {'entities': [(24, 28, '<ATTR>'), (33, 40, '<MACHINE>'), (12, 20, '<DIRECTION>')]}),
('what is the decrease in oee for BOT10', {'entities': [(24, 27, '<ATTR>'), (32, 37, '<MACHINE>'), (12, 20, '<DIRECTION>')]}),
('increase in UPH for HiPO in the last 4 hours', {'entities': [(12, 15, '<ATTR>'), (20, 24, '<MACHINE>'), (32, 44, '<TIME>')]}),
('reduction in failures for hp229 yesterday', {'entities': [(13, 21, '<ATTR>'), (26, 31, '<MACHINE>'), (32, 41, '<DATE>')]}),
('lowest UPH for aeroscope yesterday', {'entities': [(7, 10, '<ATTR>'), (15, 24, '<MACHINE>'), (25, 34, '<DATE>'), (0, 6, '<DESC>')]}),
('DS209 highest total engineering time last week', {'entities': [(14, 36, '<ATTR>'), (0, 5, '<MACHINE>'), (37, 46, '<DATE>'), (6, 13, '<DESC>')]}),
('highest mfe yesterday for iHAWK', {'entities': [(8, 11, '<ATTR>'), (26, 31, '<MACHINE>'), (12, 21, '<DATE>'), (0, 7, '<DESC>')]}),
('which is600gs has the lowest tet yesterday', {'entities': [(29, 32, '<ATTR>'), (6, 13, '<MACHINE>'), (33, 42, '<DATE>')]}),
('what is the number of is600gs with low TUDT', {'entities': [(39, 43, '<ATTR>'), (22, 29, '<MACHINE>')]}),
('greatest number of failures from ms60 yesterday', {'entities': [(9, 27, '<ATTR>'), (33, 37, '<MACHINE>'), (38, 47, '<DATE>'), (0, 8, '<DESC>')]}),
('why reduction in OEE in AD90 today', {'entities': [(17, 20, '<ATTR>'), (24, 28, '<MACHINE>'), (29, 34, '<DATE>'), (4, 13, '<DIRECTION>')]}),
('how to increase oee for ad90', {'entities': [(16, 19, '<ATTR>'), (24, 28, '<MACHINE>'), (7, 15, '<DIRECTION>')]}),
('what is the decrease in TNDT RO832 last week', {'entities': [(24, 28, '<ATTR>'), (29, 34, '<MACHINE>'), (35, 44, '<DATE>'), (12, 20, '<DIRECTION>')]}),
('drop in uph of ihb139 yesterday', {'entities': [(8, 11, '<ATTR>'), (15, 21, '<MACHINE>'), (22, 31, '<DATE>'), (0, 4, '<DIRECTION>')]}),
('why OEE decrease for SD832D today', {'entities': [(4, 7, '<ATTR>'), (21, 27, '<MACHINE>'), (28, 33, '<DATE>'), (8, 16, '<DIRECTION>')]}),
('what caused output of stk40 to fall', {'entities': [(12, 18, '<ATTR>'), (22, 27, '<MACHINE>'), (31, 35, '<DIRECTION>')]}),
('reason for reduction in oee for stk40 yesterday', {'entities': [(24, 27, '<ATTR>'), (32, 37, '<MACHINE>'), (38, 47, '<DATE>'), (11, 20, '<DIRECTION>')]}),
('how much did oee increase for ab380 last week', {'entities': [(13, 16, '<ATTR>'), (30, 35, '<MACHINE>'), (36, 45, '<DATE>'), (17, 25, '<DIRECTION>')]}),
('why did uph fall for AD90', {'entities': [(8, 11, '<ATTR>'), (21, 25, '<MACHINE>'), (12, 16, '<DIRECTION>')]}),
('which ab380 has the lowest oee', {'entities': [(27, 30, '<ATTR>'), (6, 11, '<MACHINE>'), (20, 26, '<DESC>')]}),
('What is output of 3rdbase', {'entities': [(8, 14, '<ATTR>'), (18, 25, '<MACHINE>')]}),
('mean time between assists AB350', {'entities': [(0, 25, '<ATTR>'), (26, 31, '<MACHINE>')]}),
('AB380 total productive time', {'entities': [(6, 27, '<ATTR>'), (0, 5, '<MACHINE>')]}),
('total engineering time AB339', {'entities': [(0, 22, '<ATTR>'), (23, 28, '<MACHINE>')]}),
('total nonscheduled down time ab383', {'entities': [(0, 28, '<ATTR>'), (29, 34, '<MACHINE>')]}),
('total standby time ab339', {'entities': [(0, 18, '<ATTR>'), (19, 24, '<MACHINE>')]}),
('ab350vlamp total engineering time', {'entities': [(11, 33, '<ATTR>'), (0, 10, '<MACHINE>')]}),
('mean time between failure ab350vlamp', {'entities': [(0, 25, '<ATTR>'), (26, 36, '<MACHINE>')]}),
('ab339 manufacturing facilitation efficiency', {'entities': [(6, 43, '<ATTR>'), (0, 5, '<MACHINE>')]}),
('mfe of 3rdbase', {'entities': [(0, 3, '<ATTR>'), (7, 14, '<MACHINE>')]}),
('tpt of ab350vm', {'entities': [(0, 3, '<ATTR>'), (7, 14, '<MACHINE>')]}),
('what is the UPH of ab350vs', {'entities': [(12, 15, '<ATTR>'), (19, 26, '<MACHINE>')]}),
('total nonscheduled down time of ab383dh', {'entities': [(0, 28, '<ATTR>'), (32, 39, '<MACHINE>')]}),
('total productive time ab383dh', {'entities': [(0, 21, '<ATTR>'), (22, 29, '<MACHINE>')]}),
('ad210 total scheduled down time', {'entities': [(6, 31, '<ATTR>'), (0, 5, '<MACHINE>')]}),
('total output ab280plus', {'entities': [(6, 12, '<ATTR>'), (13, 22, '<MACHINE>')]}),
('ad280 number of failures', {'entities': [(6, 24, '<ATTR>'), (0, 5, '<MACHINE>')]}),
('ab380 number of assists', {'entities': [(6, 23, '<ATTR>'), (0, 5, '<MACHINE>')]}),
('ab383dh unit per hour', {'entities': [(8, 21, '<ATTR>'), (0, 7, '<MACHINE>')]}),
('ab350vlamp tpt', {'entities': [(11, 14, '<ATTR>'), (0, 10, '<MACHINE>')]}),
('ad210plus tet', {'entities': [(10, 13, '<ATTR>'), (0, 9, '<MACHINE>')]}),
('mtba of ad211', {'entities': [(0, 4, '<ATTR>'), (8, 13, '<MACHINE>')]}),
('what is the tudt of ad50', {'entities': [(12, 16, '<ATTR>'), (20, 24, '<MACHINE>')]}),
('tst ab350vs', {'entities': [(0, 3, '<ATTR>'), (4, 11, '<MACHINE>')]}),
('3rdbase mtbf', {'entities': [(8, 12, '<ATTR>'), (0, 7, '<MACHINE>')]}),
('mfe of 2d211plus', {'entities': [(0, 3, '<ATTR>'), (7, 16, '<MACHINE>')]}),
('ad280 output', {'entities': [(6, 12, '<ATTR>'), (0, 5, '<MACHINE>')]}),
('number of failures of ab383dh', {'entities': [(0, 18, '<ATTR>'), (22, 29, '<MACHINE>')]}),
('what is the tsdt of ab350vm', {'entities': [(12, 16, '<ATTR>'), (20, 27, '<MACHINE>')]}),
('tet of ad8312fc', {'entities': [(0, 3, '<ATTR>'), (7, 15, '<MACHINE>')]}),
('what is the mfe of ad50plus', {'entities': [(12, 15, '<ATTR>'), (19, 27, '<MACHINE>')]}),
('mean time between assists ad828', {'entities': [(0, 25, '<ATTR>'), (26, 31, '<MACHINE>')]}),
('number of failures of ad830', {'entities': [(0, 18, '<ATTR>'), (22, 27, '<MACHINE>')]}),
('output of ad8312', {'entities': [(0, 6, '<ATTR>'), (10, 16, '<MACHINE>')]}),
('ad830l total unscheduled down time', {'entities': [(7, 34, '<ATTR>'), (0, 6, '<MACHINE>')]}),
('ad830r total productive time', {'entities': [(7, 28, '<ATTR>'), (0, 6, '<MACHINE>')]}),
('ad8312fc mtbf this monday', {'entities': [(9, 13, '<ATTR>'), (0, 8, '<MACHINE>'), (14, 25, '<DATE>')]}),
('what is the tndt for ab280', {'entities': [(12, 16, '<ATTR>'), (21, 26, '<MACHINE>')]}),
('what is the number of assists ad8312', {'entities': [(12, 29, '<ATTR>'), (30, 36, '<MACHINE>')]}),
('assists for ad828', {'entities': [(0, 7, '<ATTR>'), (12, 17, '<MACHINE>')]}),
('output of ad211plus', {'entities': [(0, 6, '<ATTR>'), (10, 19, '<MACHINE>')]}),
('ad830u tst', {'entities': [(7, 10, '<ATTR>'), (0, 6, '<MACHINE>')]}),
('total standby time ad830ur', {'entities': [(0, 18, '<ATTR>'), (19, 26, '<MACHINE>')]}),
('what is the number of failures for ad838lplus', {'entities': [(12, 30, '<ATTR>'), (35, 45, '<MACHINE>')]}),
('Total engineering time of ad868', {'entities': [(0, 22, '<ATTR>'), (26, 31, '<MACHINE>')]}),
('ad8912sd total productive time', {'entities': [(9, 30, '<ATTR>'), (0, 8, '<MACHINE>')]}),
('what is the tndt ad838p', {'entities': [(12, 16, '<ATTR>'), (17, 23, '<MACHINE>')]}),
('VIM300 failures', {'entities': [(7, 15, '<ATTR>'), (0, 6, '<MACHINE>')]}),
('mtbf of HARRIER XT', {'entities': [(0, 4, '<ATTR>'), (8, 18, '<MACHINE>')]}),
('what is the greatest increase in BFMF for RO832 4 hours ago', {'entities': [(33, 37, '<ATTR>'), (42, 47, '<MACHINE>'), (48, 59, '<TIME>'), (12, 20, '<DESC>'), (21, 29, '<DIRECTION>')]}),
('top 5 ms100 with highest oee last monday', {'entities': [(25, 28, '<ATTR>'), (6, 11, '<MACHINE>'), (29, 40, '<DATE>'), (17, 24, '<DESC>'), (4, 5, '<QUANTITY>')]}),
('ORCAS with most BHCC today', {'entities': [(16, 20, '<ATTR>'), (0, 5, '<MACHINE>'), (21, 26, '<DATE>'), (11, 15, '<DESC>')]}),
('3 Skyhawk with most BUCT 2 hours ago', {'entities': [(20, 24, '<ATTR>'), (2, 9, '<MACHINE>'), (25, 36, '<TIME>'), (15, 19, '<DESC>'), (0, 1, '<QUANTITY>')]}),
('how many WINWB have more than 5 wire clamp operations last week', {'entities': [(32, 53, '<ATTR>'), (9, 14, '<MACHINE>'), (54, 63, '<DATE>'), (20, 29, '<DESC>'), (30, 31, '<QUANTITY>')]}),
('AD90 with the longest TUDT yesterday', {'entities': [(22, 26, '<ATTR>'), (0, 4, '<MACHINE>'), (27, 36, '<DATE>'), (14, 21, '<DESC>')]}),
('select top 5 ab380 with most number of failures last week', {'entities': [(29, 47, '<ATTR>'), (13, 18, '<MACHINE>'), (48, 57, '<DATE>'), (24, 28, '<DESC>'), (11, 12, '<QUANTITY>')]}),
('5 ab380 with tst more than 40min', {'entities': [(13, 16, '<ATTR>'), (2, 7, '<MACHINE>'), (17, 26, '<DESC>'), (27, 32, '<MEASUREMENT>'), (0, 1, '<QUANTITY>')]}),
('all ad90 tndt more than 90min', {'entities': [(9, 13, '<ATTR>'), (4, 8, '<MACHINE>'), (14, 23, '<DESC>'), (24, 29, '<MEASUREMENT>')]}),
('what is the lowest UPH for MC609H last week', {'entities': [(19, 22, '<ATTR>'), (27, 33, '<MACHINE>'), (34, 43, '<DATE>'), (12, 18, '<DESC>')]}),
('ISPLASMA OEE decrease by how much from 2 hours ago', {'entities': [(9, 12, '<ATTR>'), (0, 8, '<MACHINE>'), (39, 50, '<TIME>'), (13, 21, '<DIRECTION>')]}),
('how much did IS838LA3 tet increase last week', {'entities': [(22, 25, '<ATTR>'), (13, 21, '<MACHINE>'), (35, 44, '<DATE>'), (26, 34, '<DIRECTION>')]}),
('how much IDEALMOLD tpt fall from yesterday to today', {'entities': [(19, 22, '<ATTR>'), (9, 18, '<MACHINE>'), (33, 42, '<DATE>'), (46, 51, '<DATE>')]}),
('which cb830 uph decrease by 20% today', {'entities': [(12, 15, '<ATTR>'), (6, 11, '<MACHINE>'), (32, 37, '<DATE>'), (16, 24, '<DIRECTION>'), (28, 31, '<MEASUREMENT>')]}),
('how much did mtbf drop for COG902 in the last 20 mins', {'entities': [(13, 17, '<ATTR>'), (27, 33, '<MACHINE>'), (41, 53, '<DATE>'), (18, 22, '<DIRECTION>')]}),
('5 CS8KAP with assists higher than 50%', {'entities': [(14, 21, '<ATTR>'), (2, 8, '<MACHINE>'), (22, 33, '<DESC>'), (34, 37, '<MEASUREMENT>'), (0, 1, '<QUANTITY>')]}),
('which cm868s have tndt greater than 30mins', {'entities': [(18, 22, '<ATTR>'), (6, 12, '<MACHINE>'), (23, 35, '<DESC>'), (36, 42, '<MEASUREMENT>')]}),
('number of aerocami that output less than 30%', {'entities': [(24, 30, '<ATTR>'), (10, 18, '<MACHINE>'), (31, 40, '<DESC>'), (41, 44, '<MEASUREMENT>')]}),
('what is the reduction in oee of aerogauge on wednesday', {'entities': [(25, 28, '<ATTR>'), (32, 41, '<MACHINE>'), (45, 54, '<DATE>'), (12, 21, '<DIRECTION>')]}),
('drop in uph of ihb139 yesterday', {'entities': [(8, 11, '<ATTR>'), (15, 21, '<MACHINE>'), (22, 31, '<DATE>'), (0, 4, '<DIRECTION>')]}),
('how many ihb139 uph drop below 30', {'entities': [(16, 19, '<ATTR>'), (9, 15, '<MACHINE>'), (20, 24, '<DIRECTION>'), (31, 33, '<MEASUREMENT>')]}),
('what is the increase in mtba of ab380', {'entities': [(24, 28, '<ATTR>'), (32, 37, '<MACHINE>'), (12, 20, '<DIRECTION>')]}),
('change in mtbf of at420 yesterday', {'entities': [(10, 14, '<ATTR>'), (18, 23, '<MACHINE>'), (24, 33, '<DATE>'), (0, 6, '<DIRECTION>')]}),
('MTBF of EAGLE AR increase by how much', {'entities': [(0, 4, '<ATTR>'), (8, 16, '<MACHINE>'), (17, 25, '<DIRECTION>')]}),
('which IDEALMOLD oee increase more than 20% from monday to tuesday', {'entities': [(16, 19, '<ATTR>'), (6, 15, '<MACHINE>'), (48, 54, '<DATE>'), (58, 65, '<DATE>'), (29, 38, '<DESC>'), (20, 28, '<DIRECTION>'), (39, 42,'<MEASUREMENT>')]}),
('number of ms90 tst less than 30min', {'entities': [(15, 18, '<ATTR>'), (10, 14, '<MACHINE>'), (19, 28, '<DESC>'), (29, 34, '<MEASUREMENT>')]}),
('number of ps410 with tsdt more than 20min', {'entities': [(21, 25, '<ATTR>'), (10, 15, '<MACHINE>'), (26, 35, '<DESC>'), (36, 41, '<MEASUREMENT>')]}),
('number of tlb139 with tpt more than 2hours', {'entities': [(22, 25, '<ATTR>'), (10, 16, '<MACHINE>'), (26, 35, '<DESC>'), (36, 42, '<MEASUREMENT>')]})
]

# Switch off other components in the spacy pipeline
# Leave the Named Entity Recognition component active for training

In [8]:
for _, annotations in TRAINING_DATA:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

# Start Training

In [9]:
# optimizer = nlp.initialize()
with nlp.disable_pipes(*unaffected_pipes):
    optimizer = nlp.create_optimizer()
    for iteration in range(15):
        random.shuffle(TRAINING_DATA)
        losses = {}
        
        # batches = minibatch(TRAINING_DATA, size=compounding(4.0, 32,1.001))
        for texts, annotations in TRAINING_DATA:
            # texts, annotations = zip(*batch)
            # print(texts)
            # print(annotations)

            doc = nlp.make_doc(texts)

            example = Example.from_dict(doc, annotations)
            nlp.update(
                [example],
                drop=0.5,
                losses=losses,
                sgd=optimizer
            )
            print("losses", losses)


losses {'ner': 33.96542480247317}
losses {'ner': 35.78358295658621}
losses {'ner': 35.82380760133091}
losses {'ner': 37.81266024549319}
losses {'ner': 37.812739133861825}
losses {'ner': 37.81273915091079}
losses {'ner': 37.826543855442566}
losses {'ner': 37.82654395511032}
losses {'ner': 37.82654395511062}
losses {'ner': 37.82654395654329}
losses {'ner': 37.82655878022183}
losses {'ner': 37.82655878022221}
losses {'ner': 37.826558784404845}
losses {'ner': 37.826583604480696}
losses {'ner': 41.91496062481218}
losses {'ner': 43.98083534191613}
losses {'ner': 43.98085207662058}
losses {'ner': 43.9895099958636}
losses {'ner': 44.0870396878568}
losses {'ner': 46.27723610825687}
losses {'ner': 46.27724280250942}
losses {'ner': 47.848147366942}
losses {'ner': 47.84814744945686}
losses {'ner': 47.956224629368336}
losses {'ner': 47.95622587716957}
losses {'ner': 47.956225904206505}
losses {'ner': 47.95622590431496}
losses {'ner': 47.9562872743693}
losses {'ner': 47.95628732077019}
losses {'ner

# Method to test the model
## Type 'quit' to escape

In [13]:
while True:
    tokens = []
    user_input = input()
    if user_input == 'quit':
        break
    doc = nlp(user_input)
    # for ent in doc.ents:
    displacy.render(doc, style="ent")
    print(doc)
    for d in doc:
        print(d.text, d.ent_type_)

how many ab380 have tst more than equal to 0
how 
many 
ab380 <MACHINE>
have 
tst <ATTR>
more <DESC>
than <DESC>
equal <DIRECTION>
to 
0 <QUANTITY>


how many ab380 tst >= 0
how 
many 
ab380 <MACHINE>
tst <ATTR>
> 
= 
0 <TIME>


# save model to file

In [11]:
# nlp.to_disk('TOKENIZER_NER_ATTR_MACHINE_DATE_TIME_DESC_DIR_MEASUREMENT_QUANTITY')